# L2: How To Use Structured Outputs

+ Pros
    + JSON is a very powerful form of structured outputs!
    + Easy to use if already using OpeanAI or most other proprietary APIs
    + Rapidly evolving space with new features popping up frequently
+ Cons
    + Only works with specific, closed models
    + Changing providers often requires a major refactor
    + Inconsistent results (depending on provider)
    + Unclear impact on quality of output
    + Limited options for types of structure, tipically a limited subset of JSON

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# import os
# from helper import get_openai_api_key
# KEY = get_openai_api_key()

In [3]:
from openai import OpenAI

# Instantiate the client
client = OpenAI(
    # api_key=KEY
)

## Define structure with Pydantic

In [4]:
# The user class from the slides
from pydantic import BaseModel
from typing import Optional

class User(BaseModel):
    name: str
    age: int
    email: Optional[str] = None

In [5]:
completion = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Make up a user."},
    ],
    response_format=User,
)

In [7]:
completion

ParsedChatCompletion[User](id='chatcmpl-BItm6ybyxlLMkbRViLRKIYsmR0BL5', choices=[ParsedChoice[User](finish_reason='stop', index=0, logprobs=None, message=ParsedChatCompletionMessage[User](content='{"name":"John Doe","age":28,"email":"johndoe@example.com"}', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[], parsed=User(name='John Doe', age=28, email='johndoe@example.com'), annotations=[]))], created=1743844002, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_b376dfbbd5', usage=CompletionUsage(completion_tokens=20, prompt_tokens=85, total_tokens=105, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [6]:
user = completion.choices[0].message.parsed
user

User(name='John Doe', age=28, email='johndoe@example.com')

## The social media mention structure

In [8]:
from pydantic import BaseModel
from enum import Enum
from typing import List, Optional, Literal
from openai import OpenAI

class Mention(BaseModel):
    # The model chooses the product the mention is about,
    # as well as the social media post's sentiment
    product: Literal['app', 'website', 'not_applicable']
    sentiment: Literal['positive', 'negative', 'neutral']

    # Model can choose to respond to the user
    needs_response: bool
    response: Optional[str]

    # If a support ticket needs to be opened, 
    # the model can write a description for the
    # developers
    support_ticket_description: Optional[str]

In [9]:
# Example mentions
mentions = [
    # About the app
    "@techcorp your app is amazing! The new design is perfect",
    # Website is down, negative sentiment + needs a fix
    "@techcorp website is down again, please fix!",
    # Nothing to respond to
    "hey @techcorp you're so evil"
]

In [10]:
def analyze_mention(
    mention: str, 
    personality: str = "friendly"
) -> Mention:
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": f"""
                Extract structured information from 
                social media mentions about our products.

                Provide
                - The product mentioned (website, app, not applicable)
                - The mention sentiment (positive, negative, neutral)
                - Whether to respond (true/false). Don't respond to 
                  inflammatory messages or bait.
                - A customized response to send to the user if we need 
                  to respond.
                - An optional support ticket description to create.

                Your personality is {personality}.
            """},
            {"role": "user", "content": mention},
        ],
        response_format=Mention,
    )
    return completion.choices[0].message.parsed

In [11]:
print("User post:", mentions[0])
processed_mention = analyze_mention(mentions[0])
processed_mention

User post: @techcorp your app is amazing! The new design is perfect


Mention(product='app', sentiment='positive', needs_response=True, response="Thank you so much for your kind words! We're thrilled to hear that you love the new design. If you have any feedback or suggestions, feel free to share!", support_ticket_description=None)

In [12]:
rude_mention = analyze_mention(mentions[0], personality="rude")
rude_mention.response

'Well, finally someone with good taste! Thanks for noticing the changes, enjoy!'

In [13]:
mention_json_string = processed_mention.model_dump_json(indent=2)
print(mention_json_string)

{
  "product": "app",
  "sentiment": "positive",
  "needs_response": true,
  "response": "Thank you so much for your kind words! We're thrilled to hear that you love the new design. If you have any feedback or suggestions, feel free to share!",
  "support_ticket_description": null
}


## You try!

In [14]:
class UserPost(BaseModel):
    message: str

def make_post(output_class):
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": """
                You are a customer of Tech Corp (@techcorp), a company
                that provides an app and a website. Create a small 
                microblog-style post to them that sends some kind of 
                feedback, positive or negative.
            """},
            {"role": "user", "content": "Please write a post."},
        ],
        response_format=output_class,
    )
    return completion.choices[0].message.parsed

new_post = make_post(UserPost)
new_post

UserPost(message='Just wanted to say how much I love the Tech Corp app! The user interface is sleek and easy to navigate, and I appreciate the regular updates that keep improving the experience. Keep up the great work!')

In [15]:
analyze_mention(new_post.message)

Mention(product='app', sentiment='positive', needs_response=True, response="Thank you so much for your kind words! We're thrilled to hear that you love the Tech Corp app and appreciate the updates. Your feedback motivates us to keep improving. If you have any suggestions or ideas, feel free to share!", support_ticket_description=None)

## Programming with our mentions

In [16]:
from helper import print_mention

# Loop through posts that tagged us and store the results in a list
rows = []
for mention in mentions:
    # Call the LLM to get a Mention object we can program with
    processed_mention = analyze_mention(mention)

    # Print out some information
    print_mention(processed_mention, mention)
    
    # Convert our processed data to a dictionary
    # using Pydantic tools
    processed_dict = processed_mention.model_dump()
    
    # Store the original message in the dataframe row
    processed_dict['mention'] = mention
    rows.append(processed_dict)
    
    print("") # Add separator to make it easier to read

Responding to positive app feedback
  User: @techcorp your app is amazing! The new design is perfect
  Response: Thank you so much for your kind words! We're thrilled to hear you love the new design! If you have any more feedback or suggestions, feel free to share!

Responding to negative website feedback
  User: @techcorp website is down again, please fix!
  Response: We're sorry to hear you're experiencing issues with our website! We're on it and will get it back up as soon as possible. Thank you for your patience!
  Adding support ticket: User reported that the website is down. Request to investigate the cause and restore functionality.

Not responding to negative not_applicable post
  User: hey @techcorp you're so evil

